# Procédure de la récupération des données Météo France SWI 1969—2022

## Les données d'intérêt

### 1. Les données Météo France

Se rendre à la [page dédiée][1] à ce jeu de données. Récupérer :

- ✅ L'archive ZIP avec les données de mesure (date AAAAMM, numéro maille, mesure) depuis la rubrique `Téléchargement`.

- ❌ Le fichier `metadonnees_swi_276.csv` (numéro maille, latitude, longitude) depuis la rubrique `Documentation` > `Coordonnées des mailles`.
  
  Néanmoins, après étude, le référentiel `metadonnees_swi_276.csv` s'est avéré comme insuffisamment informatif. Par conséquent, à l'issue de recherches supplémentaires, nous avons décidé de nous tourner vers un fichier équivalent, mais plus complet, édité par l'INRAE.

### 2. Les données INRAE

- ✅ Le fichier `mailles_safran_drias-20200206.csv` de l'INRAE ([source][2]) comporte au moins deux avantages par rapport au fichier `metadonnees_swi_276.csv` de Météo France :

  - les coordonnées (longitude, latitude) sont plus précises (12 chiffres significatifs au lieu de 6) ;

  - sélection des mailles selon des critères administratifs (département, région), indispensables pour notre projet.

[1]: https://donneespubliques.meteofrance.fr/?fond=produit&id_produit=301&id_rubrique=40
[2]: https://entrepot.recherche.data.gouv.fr/file.xhtml?persistentId=doi:10.57745/1PDFNL/L7PDV7&version=2.0

### Les données du fichier `mailles_safran_drias-20200206.csv`

Certains champs vont devoir être supprimés, renommés ou transformés.

Légende :

| Que faire ? | Colonne | Type | Description |
| --- | --- | --- | --- |
|  | maille_safran |  |  |
| supprimer | maille_drias |  |  |
| renommer | latitude |  |  |
| renommer | longitude |  |  |
|  | altitude | NUMERIC | Altitude moyenne de la maille (m). |
| transformer | departement | VARCHAR | Département. |
|  | region_avant_2016 | VARCHAR | Région avant 2016. |
|  | region2016 | VARCHAR | Région. |
| supprimer | nature | VARCHAR | Localisation de la maille («», « Territoire », « Limite côtière », « Frontière internationale »). |
| supprimer | clc_211 | NUMERIC | Pourcentage de la maille en « Terres arables hors périmètres d''irrigation » (CLC 211). |
| supprimer | clc_221 | NUMERIC | Pourcentage de la maille en « Vignoble » (CLC 221). |
| supprimer | clc_222 | NUMERIC | Pourcentage de la maille en « Vergers et petits fruits » (CLC 222). |
| supprimer | clc_231 | NUMERIC | Pourcentage de la maille en « Prairies » (CLC 231). |
| supprimer | clc_241 | NUMERIC | Pourcentage de la maille en « Cultures annuelles associées aux cultures permanentes » (CLC 241). |
| supprimer | clc_242 | NUMERIC | Pourcentage de la maille en « Systèmes culturaux et parcellaires complexes » (CLC 242). |
| supprimer | clc_311 | NUMERIC | Pourcentage de la maille en « Forêts de feuillus » (CLC 311). |
| supprimer | clc_312 | NUMERIC | Pourcentage de la maille en « Forêts de conifères » (CLC 312). |
| supprimer | clc_313 | NUMERIC | Pourcentage de la maille en « Forêts mélangées » (CLC 313). |
| supprimer | rupro | NUMERIC | Réserve utile (mm). |
| supprimer | rupromin | NUMERIC | Réserve utile minimale (mm). |
| supprimer | rupromax | NUMERIC | Réserve utile maximale (mm). |
| supprimer | pro | NUMERIC | Profondeur de sol (cm). |
| supprimer | hpf | NUMERIC | Humidité au point de flétrissement (%). |
| supprimer | hcc | NUMERIC | Humidité à la capacité au champ (%). |
| supprimer | hsat | NUMERIC | Humidité à saturation (%). |

In [20]:
from datetime import datetime
from google.cloud import storage as GCS
from tqdm import tqdm
import zipfile
import io
import os
import pandas as pd
import requests
import sys

### Préparer le fichier `mailles_safran_drias-20200206.csv`

In [2]:
def split_dep_en_code_et_nom(df):
    dep = 'departement'
    code_et_nom = ['code_departement', 'nom_departement']
    re_pattern = r'^(\w{2}) - (.+)$'
    df[code_et_nom] = df[dep].str.extract(re_pattern)
    return df


In [3]:
# df_geo_coords = pd.read_csv('mailles_safran_drias-20200206.csv')
df_geo_coords = pd.read_csv('gs://{bucket_name}/{folder_name}/{file_name}'.format(
    bucket_name='code_de_source_lake',
    folder_name='meteofrance-swi',
    file_name='mailles_safran_drias-20200206.csv'))
df_geo_coords = split_dep_en_code_et_nom(df_geo_coords)
colonnes_à_suppr = ['maille_drias', 'departement', 'nature', 'clc_211',
                    'clc_221', 'clc_222', 'clc_231', 'clc_241', 'clc_242',
                    'clc_311', 'clc_312', 'clc_313',
                    'rupro', 'rupromin', 'rupromax',
                    'pro', 'hpf', 'hcc', 'hsat']
df_geo_coords.drop(colonnes_à_suppr, axis='columns', inplace=True)
colonnes_à_renommer = {'maille_safran': 'num_maille',
                       'longitude': 'longitude_x',
                       'latitude': 'latitude_y'}
df_geo_coords.rename(columns=colonnes_à_renommer, inplace=True)
ordered_columns = ['num_maille', 'latitude_y', 'longitude_x', 'altitude',
                   'code_departement', 'nom_departement',
                   'region_avant_2016', 'region2016']
df_geo_coords = df_geo_coords[ordered_columns]
df_geo_coords


,num_maille,latitude_y,longitude_x,altitude,code_departement,nom_departement,region_avant_2016,region2016
0,4,51.049736,2.393384,2,59,Nord,Nord-Pas-de-Calais,Hauts-de-France
1,5,51.049634,2.507161,2,59,Nord,Nord-Pas-de-Calais,Hauts-de-France
2,13,50.977389,1.938856,4,62,Pas-de-Calais,Nord-Pas-de-Calais,Hauts-de-France
3,14,50.977702,2.052466,3,62,Pas-de-Calais,Nord-Pas-de-Calais,Hauts-de-France
4,15,50.977910,2.166078,2,59,Nord,Nord-Pas-de-Calais,Hauts-de-France
...,...,...,...,...,...,...,...,...
8597,9885,41.492497,8.975897,182,2A,Corse-du-sud,Corse,Corse
8598,9886,41.486401,9.070964,64,2A,Corse-du-sud,Corse,Corse
8599,9887,41.480219,9.166011,167,2A,Corse-du-sud,Corse,Corse
8600,9888,41.473950,9.261037,102,2A,Corse-du-sud,Corse,Corse


### Télécharger l'archive ZIP

In [4]:
def download_zip_archive(url: str, target_file_name: str) -> bytes:
    with requests.get(url, stream=True) as response:
        response.raise_for_status()
        total_size = int(response.headers.get('content-length', 0))
        tqdm_params = {'desc': target_file_name,
                        'total': total_size,
                        'miniters': 1,
                        'unit': 'B',
                        'unit_scale': True,
                        'unit_divisor': 1024}
        content_as_bytes = b''
        with tqdm(**tqdm_params) as progr_bar:
            for chunk in response.iter_content(chunk_size=4096):
                progr_bar.update(len(chunk))
                content_as_bytes += chunk
        return content_as_bytes
    return None


In [5]:
première_année_archivée = 1969
année_en_cours = datetime.now().year
dernière_année_archivée = année_en_cours - 1
zip_file_name = 'SWI_Package_{0}-{1}.zip'.format(première_année_archivée,
                                                 dernière_année_archivée)
url = f'https://donneespubliques.meteofrance.fr/donnees_libres/Txt/Swi/{zip_file_name}'
zip_archive_bytes = download_zip_archive(url=url, target_file_name=zip_file_name)


SWI_Package_1969-2022.zip: 100%|██████████| 26.3M/26.3M [01:03<00:00, 437kB/s] 


### Compléter chacun des fichiers CSV de l'archive ZIP avec les infos géographiques et administratives issues du fichier `mailles_safran_drias-20200206.csv`

In [48]:
def complete_df_with_geo_coords(df_without_geo_coords: pd.DataFrame,
                                df_geo_coords: pd.DataFrame) -> pd.DataFrame:
    df_with_geo_coords = pd.merge(df_without_geo_coords, df_geo_coords,
                                  how='inner',
                                  left_on=['NUMERO'],
                                  right_on=['num_maille'])
    df_with_geo_coords.convert_dtypes(infer_objects=False)
    df_with_geo_coords.drop(['NUMERO', 'LAMBX', 'LAMBY'],
                            axis=1, inplace=True)
    colonnes_à_renommer = {'lat_dg': 'latitude_y',
                            'lon_dg': 'longitude_x',
                            'SWI_UNIF_MENS3': 'soil_wetness_index_monthly_avg',
                            'DATE': 'date'}
    df_with_geo_coords.rename(columns=colonnes_à_renommer, inplace=True)
    df_with_geo_coords['date'] = pd.to_datetime(df_with_geo_coords['date'],
                                                format='%Y%m')
    return df_with_geo_coords


def save_to_GCS(file_content_str: str,
                target_file_name: str,
                bucket_name: str,
                folder_name: str = '.'):
    storage_client = GCS.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(f'{folder_name}/{target_file_name}')
    blob.upload_from_string(file_content_str)


def sort_naturally(lst: list) -> list:
    max_str_len = max([len(s) for s in lst])
    return sorted(lst, key=lambda s: s.zfill(max_str_len + 1))

In [51]:
zip_buffer = io.BytesIO(zip_archive_bytes)
with zipfile.ZipFile(zip_buffer, 'r') as zf:
    with tqdm(total=len(zf.filelist), ncols=100) as progr_bar:
        for csv_file in sort_naturally(zf.filelist):
            progr_bar.set_description(f"Fichier {csv_file.filename!r} : complétion")
            csv_file_bytes = zf.read(csv_file.filename)
            csv_buffer = io.BytesIO(csv_file_bytes)
            df_without_geo_coords = pd.read_csv(csv_buffer, sep=';', decimal=',')
            df_with_geo_coords = complete_df_with_geo_coords(df_without_geo_coords,
                                                             df_geo_coords)
            progr_bar.set_description(f"Fichier {csv_file.filename!r} : expédition")
            save_to_GCS(file_content_str=df_with_geo_coords.to_csv(index=False),
                        target_file_name=csv_file.filename,
                        bucket_name='code_de_source_lake',
                        folder_name='meteofrance-swi')
            progr_bar.update()

Fichier 'swi.9750-9999.csv' : expédition: 100%|█████████████████████| 40/40 [01:53<00:00,  2.84s/it]
